# 자연어 인식

In [40]:
from sklearn.model_selection import train_test_split
import re
import sys
from sklearn.externals import  joblib
from sklearn.grid_search import GridSearchCV
from sklearn.svm import  LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import  TfidfVectorizer,CountVectorizer
import os
import numpy as np
import string
import json
import nltk
import pyaudio
from konlpy.tag import *
kkma=Kkma()
tw=Twitter()
po = pyaudio.PyAudio()
for index in range(po.get_device_count()): 
    desc = po.get_device_info_by_index(index)
    #if desc["name"] == "record":
    print ("DEVICE: %s  INDEX:  %s  RATE:  %s " %  (desc["name"], index,  int(desc["defaultSampleRate"])))


DEVICE: Microsoft »ç¿îµå ¸ÅÆÛ - Input  INDEX:  0  RATE:  44100 
DEVICE: ¸¶ÀÌÅ© ¹è¿­(Realtek High Defini  INDEX:  1  RATE:  44100 
DEVICE: Microsoft »ç¿îµå ¸ÅÆÛ - Output  INDEX:  2  RATE:  44100 
DEVICE: ½ºÇÇÄ¿(Realtek High Definition   INDEX:  3  RATE:  44100 


In [76]:
import pyaudio
import wave

CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
RECORD_SECONDS = 5
WAVE_OUTPUT_FILENAME = "output.wav"

p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

print("Start to record the audio.")

frames = []

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)

print("Recording is finished.")

stream.stop_stream()
stream.close()
p.terminate()

wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()

Start to record the audio.
Recording is finished.


In [77]:
import urllib3
import json
import base64
openApiURL = "http://aiopen.etri.re.kr:8000/WiseASR/Recognition"
accessKey = "32fc3812-32a4-471a-bab0-cab768305735"
audioFilePath = "output.wav"
languageCode = "korean"
 
file = open(audioFilePath, "rb")
audioContents = base64.b64encode(file.read()).decode("utf8")
file.close()
 
requestJson = {
    "access_key": accessKey,
    "argument": {
        "language_code": languageCode,
        "audio": audioContents
    }
}
 
http = urllib3.PoolManager()
response = http.request(
    "POST",
    openApiURL,
    headers={"Content-Type": "application/json; charset=UTF-8"},
    body=json.dumps(requestJson)
)
 
print("[responseCode] " + str(response.status))
print("[responBody]")

[responseCode] 200
[responBody]


In [85]:
jsonData=json.loads(response.data.decode('utf-8'))
resultT=jsonData['return_object']['recognized']
print(resultT)

오른쪽으로 돌아


In [80]:
x_text_list=[]
f = open("x_text_list.out", 'r')
lines=f.readlines()
for line in lines:
    x_text_list.append(line.strip('\n'))
print(x_text_list)

['왼쪽 왼쪽 돌아 돋 ', '좌 좌 회전 회전 ', '좌 좌 돌아 돋 ', '왼쪽 왼쪽 회전 회전 ', '우 우로 회전 ', '우 우로 돌아 아 ', '오른쪽 오른쪽 회전 회전 ', '왼쪽 왼쪽 걸어 걷 ', '왼쪽 왼쪽 가 가 ', '왼쪽 왼쪽 전진 전진 ', '좌 좌 전진 전진 ', '오른쪽 오른쪽 걸어 걷 ', '오른쪽 오른쪽 가 가 ', '오른쪽 오른쪽 전진 전진 ', '우 우로 전진 ', '앞 앞 전진 전진 ', '앞 앞 가 가 ', '뒤 뒤 가 가 ', '뒤 뒤 걸어 걷 ', '공격 공격 ', '펀치 펀치 ', '주먹 주먹 때려 때리 ', '공격 공격 ', '때려 때리 ']


In [81]:
gsvc=joblib.load('model.pkl')
vectorizer = CountVectorizer()
vectorizer.fit_transform(x_text_list)

<24x12 sparse matrix of type '<class 'numpy.int64'>'
	with 35 stored elements in Compressed Sparse Row format>

In [82]:
arr=tw.pos(resultT)
arr2=kkma.pos(resultT)
tmp=''
for arr in arr:
    if arr[1]=='Noun' or arr[1]=='Verb':
        tmp=tmp+arr[0]+' '
for arr2 in arr2:
    if arr2[1][0]=='N' or arr2[1][0]=='V':
        tmp=tmp+arr2[0]+' '
predict=[]
predict.append(tmp)

In [83]:
Result = gsvc.predict(vectorizer.transform(predict))

In [84]:
print(Result)

[4]
